In [7]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import regressions 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from statistics import stdev
from sklearn.metrics import mean_squared_error
import scipy.stats
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split


players=["MD Shanaka", "KIC Asalanka", "BKG Mendis", "PWH de Silva", "DM de Silva", "MDKJ Perera", "AD Mathews", "WIA Fernando", "S Samarawickrama", "PHKD Mendis", "N Dickwella", "M Theekshana", "B Fernando", "A Dananjaya", "M Pathirana", "N Thushara", "JDF Vandersay", "D Madushanka"]
final_dati = pd.DataFrame()  # Final dataframe to store all the player data

driver = webdriver.Chrome()

for i in players:
    driver.get("http://www.cricmetric.com/playerstats.py?player={}&role=all&format=T20I&groupby=match&playerStatsFilters=on&start_date=2020-01-01&end_date=2021-12-31&start_over=0&end_over=9999".format(i.replace(' ','+')))
    time.sleep(3)  # Let the page load
    
    try:
        # Extracting batting stats of the player
        batting_table = driver.find_element(By.XPATH, '//*[@id="T20I-Batting"]/div/table')
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%']
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']

        # Switching to bowling stats tab
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="T20I-Bowling-tab"]'))).click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = driver.find_element(By.XPATH, '//*[@id="T20I-Bowling"]/div/table')
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ', expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']

    except:
        stats2 = pd.DataFrame({'Match': pd.Series(stats['Match'][0:1]),
                               'Overs Bowled': [0], 'Runs Given': [0],
                               'Wickets Taken': [0], 'Econ': [0],
                               'Bowl Avg': [0], 'Bowl SR': [0], '5W': [0],
                               '4s Given': [0], '6s Given': [0],
                               'Bowl Dot%': [0]})

    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    merged_stats = merged_stats.sort_values(by=['Match'])
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_dati =  pd.concat([final_dati,merged_stats])
final_dati = final_dati[final_dati['Match']!=0]

final_dati['Bowl Avg'] = np.where(final_dati['Bowl Avg']=='-',0,final_dati['Bowl Avg'])
final_dati['Bowl SR'] = np.where(final_dati['Bowl SR']=='-',0,final_dati['Bowl SR'])
final_dati= final_dati[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_dati = final_dati.replace('-',0)

final_dati



,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
0,MD Shanaka,2020_T20_002,7,8,1,87.5,0,0,0,0,...,0,0,2,0,25.0,9,9,0,0,0
1,MD Shanaka,2020_T20_003,9,9,1,100.0,0,0,0,1,...,0,0,0,0,0,2,8,0,0,0
2,MD Shanaka,2020_T20_018,2,8,1,25.0,0,0,0,0,...,0,0,0,0,0,31,24,1.0,10,1
3,MD Shanaka,2020_T20_019,31,24,0,129.2,0,0,3,1,...,6.0,0,1,0,50.0,50,44,0,0,0
4,MD Shanaka,2021_T20_054,50,44,1,113.6,1,0,3,2,...,0,0,0,0,0,8,14,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,JDF Vandersay,2021_T20_055,2,4,0,50.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0
2,JDF Vandersay,2021_T20_062,1,1,0,100.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,NaN
0,D Madushanka,2021_T20_014,9,9,0,100.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,2,4,0.0,0.0,0.0
1,D Madushanka,2021_T20_055,2,4,0,50.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0


In [9]:
stats2

,Match,Overs Bowled,Runs Given,Wickets Taken,Econ,Bowl Avg,Bowl SR,5W,4s Given,6s Given,Bowl Dot%
1,2021_T20_014,0,0,0,0,0,0,0,0,0,0


In [3]:
import pandas as pd
from flask import Flask, send_file

# Assuming final_data is your DataFrame

# Convert final_data to CSV
final_dati.to_csv('final_data.csv', index=False)

# Initialize Flask app
app = Flask(__name__)

@app.route('/download')
def download_file():
    # Send the CSV file to the user for download
    return send_file('final_data.csv', as_attachment=True)

if __name__ == '__main__':
    # Run the app
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Mar/2024 13:50:21] "GET /download HTTP/1.1" 200 -


In [ ]:

players=["Liton Das", "Mahmudullah", "Mohammad Naim", "Najmul Hossain Shanto", "Soumya Sarkar", "Mahedi Hasan", "Towhid Hridoy", "Anamul Haque", "Taskin Ahmed", "Jaker Ali", "Mustafizur Rahman", "Shoriful Islam", "Tanzim Hasan Sakib", "Taijul Islam"] 
final_date = pd.DataFrame()  # Final dataframe to store all the player data

driver = webdriver.Chrome()

for i in players:
    driver.get("http://www.cricmetric.com/playerstats.py?player={}&role=all&format=T20I&groupby=match&playerStatsFilters=on&start_date=2020-01-01&end_date=2021-12-31&start_over=0&end_over=9999".format(i.replace(' ','+')))
    time.sleep(3)  # Let the page load
    
    try:
        # Extracting batting stats of the player
        batting_table = driver.find_element(By.XPATH, '//*[@id="T20I-Batting"]/div/table')
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%']
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']

        # Switching to bowling stats tab
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="T20I-Bowling-tab"]'))).click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = driver.find_element(By.XPATH, '//*[@id="T20I-Bowling"]/div/table')
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ', expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']

    except:
        stats2 = pd.DataFrame({'Match': pd.Series(stats['Match'][0:1]),
                               'Overs Bowled': [0], 'Runs Given': [0],
                               'Wickets Taken': [0], 'Econ': [0],
                               'Bowl Avg': [0], 'Bowl SR': [0], '5W': [0],
                               '4s Given': [0], '6s Given': [0],
                               'Bowl Dot%': [0]})

    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    merged_stats = merged_stats.sort_values(by=['Match'])
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_date =  pd.concat([final_date,merged_stats])
final_date = final_date[final_date['Match']!=0]

final_date['Bowl Avg'] = np.where(final_date['Bowl Avg']=='-',0,final_date['Bowl Avg'])
final_date['Bowl SR'] = np.where(final_date['Bowl SR']=='-',0,final_date['Bowl SR'])
final_date= final_date[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_date = final_date.replace('-',0)

final_date


In [24]:
dfdd=totale
dfdd = pd.get_dummies(dfdd, columns=['Position'], drop_first=True, prefix=['Position'])
dfdd = dfdd[['Player','Match', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored', 'Balls Played', 'Out', 'Bat SR', '50', '100', '4s Scored', '6s Scored', 'Bat Dot%', 'Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'Bowl Avg', 'Bowl SR', '5W', '4s Given', '6s Given', 'Bowl Dot%', 'next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Credits', 'Selection', 'Points']]
dfdd['Selection'] = dfdd['Selection'].str.replace('%', '')
dfdd[dfdd.columns[6:]] = dfdd[dfdd.columns[6:]].astype(float)
df_model = dfdd[['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored','Balls Played','4s Scored', '6s Scored','Out', 'Bat SR','Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Selection', 'Credits','Points']]
# Add new columns
df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
# Convert 'Runs Scored' column to numeric type
df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')

# Add new columns
df_model.insert(loc=8, column='30 runs', value=(df_model['Runs Scored'] > 30).astype(int))
df_model.insert(loc=9, column='50 runs', value=(df_model['Runs Scored'] > 50).astype(int))
df_model.insert(loc=10, column='100 runs', value=(df_model['Runs Scored'] > 100).astype(int))
df_model.insert(loc=11, column='Duck', value=(df_model['Runs Scored'] == 0).astype(int))
# Insert new columns into df_model based on the values of 'Wickets Taken' column
df_model.insert(loc=18, column='3 wickets', value=np.where(df_model['Wickets Taken'] >= 3, 1, 0))
df_model.insert(loc=19, column='4 wickets', value=np.where(df_model['Wickets Taken'] >= 4, 1, 0))
df_model.insert(loc=20, column='5 wickets', value=np.where(df_model['Wickets Taken'] >= 5, 1, 0))

# Calculate points based on the provided rules
df_model['Points'] = (
    # announced
    df_model['Runs Scored'] +            # run
    df_model['4s Scored'] +              # boundary_bonus
    2 * df_model['6s Scored'] +          # six_bonus
    4 * df_model['30 runs'] +            # 30_run_bonus
    8 * (df_model['50 runs'] == 1) +     # half_century_bonus
    16 * (df_model['100 runs'] == 1) +   # century_bonus
    -2 * df_model['Duck'] +              # dismissal_for_duck
    25 * df_model['Wickets Taken'] +     # wicket
    8 * (df_model['Wickets Taken'] >= 3) +  # 3_wicket_haul
    8 * (df_model['Wickets Taken'] >= 4) +  # 4_wicket_haul
    16 * (df_model['Wickets Taken'] >= 5) + # 5_wicket_haul
        # maiden
                # catch
     # runout(DirectHit/Stumping)
      # runout(Catcher/Thrower)
    ((df_model['Overs Bowled'] >= 2) & (df_model['Econ'] < 5)) * 6 +     # economy_points
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] > 170)) * 6 + # strike_rate_points
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 150.01) & (df_model['Bat SR'] <= 170)) * 4 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 130) & (df_model['Bat SR'] <= 150)) * 2 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 60) & (df_model['Bat SR'] <= 70)) * -2 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 50) & (df_model['Bat SR'] <= 59.99)) * -4 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] < 50)) * -6
)

# Display the DataFrame with new 'Points' column
df_model.head()



df_model

C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\4133341787.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\4133341787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\4133341787.py:23: SettingWithCopyWarning: 
A value is trying to be set 

,Player,Position_BAT,Position_BOWL,Position_WK,Runs Scored,Balls Played,4s Scored,6s Scored,30 runs,50 runs,...,4 wickets,5 wickets,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Selection,Credits,Points
0,MD Shanaka,False,False,False,7,8.0,0.0,0.0,0,0,...,0,0,9.0,9.0,0.0,0.0,0.0,3.0,8.0,7.0
1,MD Shanaka,False,False,False,9,9.0,0.0,1.0,0,0,...,0,0,2.0,8.0,0.0,0.0,0.0,3.0,8.0,11.0
2,MD Shanaka,False,False,False,2,8.0,0.0,0.0,0,0,...,0,0,31.0,24.0,1.0,10.0,1.0,3.0,8.0,2.0
3,MD Shanaka,False,False,False,31,24.0,3.0,1.0,1,0,...,0,0,50.0,44.0,0.0,0.0,0.0,3.0,8.0,65.0
4,MD Shanaka,False,False,False,50,44.0,3.0,2.0,1,0,...,0,0,8.0,14.0,0.0,0.0,0.0,3.0,8.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,Tanzim Hasan Sakib,False,True,False,0,2.0,0.0,0.0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,85.0,7.0,-2.0
355,Taijul Islam,False,True,False,5,7.0,1.0,0.0,0,0,...,0,0,6.0,3.0,0.0,0.0,0.0,59.0,7.0,6.0
356,Taijul Islam,False,True,False,6,3.0,0.0,1.0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,59.0,7.0,8.0
357,Taijul Islam,False,True,False,0,1.0,0.0,0.0,0,0,...,0,0,0.0,2.0,0.0,0.0,0.0,59.0,7.0,-2.0


In [25]:

# Linear Regression
# Fitting the model and checking accuracy
df_model.fillna(0, inplace=True)
X =df_model[df_model.columns [1:-1]]
y = df_model[df_model.columns[-1]]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=9999)
points_model = LinearRegression().fit(X_train,y_train)
print('Training set accuracy:', points_model.score (X_train,y_train)) 
print('Test set accuracy:',points_model.score (X_test,y_test))

Training set accuracy: 0.9976698816518871
Test set accuracy: 0.9957816451462376


C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\3847174854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.fillna(0, inplace=True)


In [ ]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import regressions 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from statistics import stdev
from sklearn.metrics import mean_squared_error
import scipy.stats
import math

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
final_dat = pd.DataFrame()
players=["MD Shanaka", "KIC Asalanka", "BKG Mendis", "PWH de Silva", "DM de Silva", "MDKJ Perera", "AD Mathews", "WIA Fernando", "S Samarawickrama", "PHKD Mendis", "M Theekshana", "B Fernando", "A Dananjaya", "M Pathirana", "N Thushara", "JDF Vandersay", "D Madushanka"]
driver=webdriver.Chrome()
# Looping through all the players
#
for i in players[0:]:
    # Accessing the web page for the current player's stats
    driver.get("http://www.cricmetric.com/playerstats.py?player={}&role=all&format=T20I&groupby=match&start_date=2022-01-01&end_date=2024-03-08&start_over=0&end_over=9999".format(i.replace(' ','+')))
    time.sleep(3)

    try:
        # Extracting batting stats of the player
        batting_table = driver.find_element(By.XPATH, '//*[@id="T20I-Batting"]/div/table')
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%']
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']

        # Switching to bowling stats tab
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="T20I-Bowling-tab"]'))).click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = driver.find_element(By.XPATH, '//*[@id="T20I-Bowling"]/div/table')
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ', expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']

    except:
        stats2 = pd.DataFrame({'Match': pd.Series(stats['Match'][0:1]),
                               'Overs Bowled': [0], 'Runs Given': [0],
                               'Wickets Taken': [0], 'Econ': [0],
                               'Bowl Avg': [0], 'Bowl SR': [0], '5W': [0],
                               '4s Given': [0], '6s Given': [0],
                               'Bowl Dot%': [0]})

    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    merged_stats = merged_stats.sort_values(by=['Match'])
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_dat =  pd.concat([final_dat,merged_stats])
final_dat = final_dat[final_dat['Match']!=0]

final_dat['Bowl Avg'] = np.where(final_dat['Bowl Avg']=='-',0,final_dat['Bowl Avg'])
final_dat['Bowl SR'] = np.where(final_dat['Bowl SR']=='-',0,final_dat['Bowl SR'])
final_dat= final_dat[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_dat = final_dat.replace('-',0)
final_dat.fillna(0, inplace=True)
final_dat
players=["MD Shanaka", "KIC Asalanka", "BKG Mendis", "PWH de Silva", "DM de Silva", "MDKJ Perera", "AD Mathews", "WIA Fernando", "S Samarawickrama", "PHKD Mendis", "M Theekshana", "B Fernando",  "M Pathirana", "N Thushara", "D Madushanka"]
models = pd.DataFrame()

for i in players:
    player = final_dat[final_dat['Player'] == i] 
    player_new = player.dropna()
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[21:22]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs = Ridge(alpha=k) 
    next_runs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[22:23]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_balls = Ridge(alpha=k)
    next_balls.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[25:26]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_wkts = Ridge(alpha=k) 
    next_wkts.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[23:24]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_overs = Ridge(alpha=k)
    next_overs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[24:25]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs_given = Ridge(alpha=k) 
    next_runs_given.fit(X_train, y_train)
    
    latest = player.groupby('Player').tail(1)
    latest['next_runs'] = next_runs.predict(latest[latest.columns[2:11]])
    latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
    latest['next_overs'] = next_overs.predict(latest[latest.columns[11:21]])
    latest['next_runs_given'] = next_runs_given.predict(latest[latest.columns[11:21]])
    latest['next_wkts'] = next_wkts.predict(latest[latest.columns[11:21]])
    
    models = pd.concat([models,latest])

models['next_runs_given'] = round(models['next_runs_given'], 0)
models['next_overs'] = round(models['next_overs'], 0)
models['next_runs'] = round(models['next_runs'], 0)
models['next_balls'] = round(models['next_balls'], 0)
models['next_wkts'] = round(models['next_wkts'], 0)

models




In [16]:

final_data = pd.DataFrame()
players=["Liton Das", "Mahmudullah", "Mohammad Naim", "Najmul Hossain Shanto", "Soumya Sarkar", "Mahedi Hasan", "Towhid Hridoy", "Taskin Ahmed", "Jaker Ali", "Mustafizur Rahman", "Shoriful Islam","Rishad Hossain"] 
driver=webdriver.Chrome()
# Looping through all the players
#
for i in players[0:]:
    # Accessing the web page for the current player's stats
    driver.get("http://www.cricmetric.com/playerstats.py?player={}&role=all&format=T20I&groupby=match&start_date=2022-01-01&end_date=2024-03-02&end_date=2024-03-08&start_over=0&end_over=9999".format(i.replace(' ','+')))
    time.sleep(3)

    try:
        # Extracting batting stats of the player
        batting_table = driver.find_element(By.XPATH, '//*[@id="T20I-Batting"]/div/table')
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%']
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']

        # Switching to bowling stats tabA
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="T20I-Bowling-tab"]'))).click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = driver.find_element(By.XPATH, '//*[@id="T20I-Bowling"]/div/table')
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ', expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']

    except:
        stats2 = pd.DataFrame({'Match': pd.Series(stats['Match'][0:1]),
                               'Overs Bowled': [0], 'Runs Given': [0],
                               'Wickets Taken': [0], 'Econ': [0],
                               'Bowl Avg': [0], 'Bowl SR': [0], '5W': [0],
                               '4s Given': [0], '6s Given': [0],
                               'Bowl Dot%': [0]})

    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    merged_stats = merged_stats.sort_values(by=['Match'])
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_data =  pd.concat([final_data,merged_stats])
final_data = final_data[final_data['Match']!=0]

final_data['Bowl Avg'] = np.where(final_data['Bowl Avg']=='-',0,final_data['Bowl Avg'])
final_data['Bowl SR'] = np.where(final_data['Bowl SR']=='-',0,final_data['Bowl SR'])
final_data= final_data[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_data = final_data.replace('-',0)
final_data.fillna(0, inplace=True)
final_data
players=["Liton Das", "Mahmudullah", "Mohammad Naim", "Najmul Hossain Shanto", "Soumya Sarkar", "Mahedi Hasan", "Towhid Hridoy", "Taskin Ahmed", "Jaker Ali", "Mustafizur Rahman", "Shoriful Islam", "Rishad Hossain"] 
models1 = pd.DataFrame()

for i in players:
    player = final_data[final_data['Player'] == i] 
    player_new = player.dropna()
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[21:22]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs = Ridge(alpha=k) 
    next_runs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[2:11]]
    y = player_new[player_new.columns[22:23]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_balls = Ridge(alpha=k)
    next_balls.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[25:26]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_wkts = Ridge(alpha=k) 
    next_wkts.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[23:24]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_overs = Ridge(alpha=k)
    next_overs.fit(X_train, y_train)
    
    X = player_new[player_new.columns[11:21]]
    y = player_new[player_new.columns[24:25]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    ridge = pd.DataFrame()
    for j in range(101):
        points = Ridge(alpha=j).fit(X_train, y_train)
        ridge_df = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points.score(X_train, y_train)), 'Test': pd.Series(points.score(X_test, y_test))})
        ridge = pd.concat([ridge, ridge_df])
    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)
    try:
        k = ridge[ridge['Average'] == ridge['Average'].max()]['Alpha'].iloc[0]
    except IndexError:
        k = 0 
    next_runs_given = Ridge(alpha=k) 
    next_runs_given.fit(X_train, y_train)
    
    latest = player.groupby('Player').tail(1)
    latest['next_runs'] = next_runs.predict(latest[latest.columns[2:11]])
    latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
    latest['next_overs'] = next_overs.predict(latest[latest.columns[11:21]])
    latest['next_runs_given'] = next_runs_given.predict(latest[latest.columns[11:21]])
    latest['next_wkts'] = next_wkts.predict(latest[latest.columns[11:21]])
    
    models1 = pd.concat([models1,latest])

models1['next_runs_given'] = round(models1['next_runs_given'], 0)
models1['next_overs'] = round(models1['next_overs'], 0)
models1['next_runs'] = round(models1['next_runs'], 0)
models1['next_balls'] = round(models1['next_balls'], 0)
models1['next_wkts'] = round(models1['next_wkts'], 0)

models1




C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\2341726428.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest['next_runs'] = next_runs.predict(latest[latest.columns[2:11]])
C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\2341726428.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\2341726428.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
27,Liton Das,2023_T20_414,42,36,0,116.7,0,0,2,1,...,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0
7,Mahmudullah,2022_T20_247,27,22,1,122.7,0,0,1,1,...,0,0,0,0,0,13.0,13.0,0.0,2.0,0.0
2,Mohammad Naim,2022_T20_245,6,8,1,75.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,Najmul Hossain Shanto,2023_T20_416,17,15,1,113.3,0,0,4,0,...,0,0,1,1,0.0,8.0,5.0,-0.0,-0.0,0.0
7,Soumya Sarkar,2023_T20_416,4,4,1,100.0,0,0,1,0,...,0,0,0,0,0,8.0,7.0,0.0,0.0,0.0
11,Mahedi Hasan,2023_T20_416,4,13,1,30.8,0,0,0,0,...,12.0,0,2,0,58.3,0.0,-1.0,1.0,7.0,0.0
8,Towhid Hridoy,2023_T20_416,16,18,1,88.9,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,13.0,6.0,0.0,0.0,0.0
8,Anamul Haque,2023_T20_416,16,18,1,88.9,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,13.0,6.0,0.0,0.0,0.0
20,Taskin Ahmed,2023_T20_095,0,0,0,0,0,0,0,0,...,7.0,0,2,4,67.9,4.0,2.0,1.0,9.0,1.0
1,Jaker Ali,2023_T20_282,24,29,0,82.8,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,24.0,29.0,0.0,0.0,0.0


In [18]:
dfcf = pd.read_csv('OP2.csv')
fdfs=pd.concat([models,models1])
# Merge the CSV data with the existing DataFrame based on the "Name" column in CSV and "Player" column in the existing DataFrame
totali = pd.merge(fdfs, dfcf, left_on='Player', right_on='Name', how='left')

# Drop the redundant "Name" column
totali.drop(columns=['Name'], inplace=True)
totali =totali.rename(columns={'Selected_By': 'Selection'})

# Display the merged DataFrame
totali.head()
totali=totali[['Player','Match','Runs Scored','Balls Played','Out','Bat SR','50','100','4s Scored','6s Scored','Bat Dot%','Overs Bowled','Runs Given','Wickets Taken','Econ','Bowl Avg','Bowl SR','5W','4s Given','6s Given','Bowl Dot%','next_runs', 'next_balls', 'next_overs','next_runs_given', 'next_wkts','Team','Position','Credits','Selection','Points']]
totali

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Team,Position,Credits,Selection,Points
0,MD Shanaka,2024_T20_023,20,18,0,111.1,0,0,1,1,...,15.0,13.0,1.0,9.0,1.0,SL,AR,8.0,3%,0
1,KIC Asalanka,2024_T20_023,28,14,1,200.0,0,0,1,3,...,24.0,19.0,0.0,1.0,0.0,SL,BAT,8.5,8%,0
2,BKG Mendis,2024_T20_023,36,22,1,163.6,0,0,2,3,...,27.0,18.0,0.0,0.0,0.0,SL,WK,8.5,70%,0
3,PWH de Silva,2024_T20_007,0,0,0,0,0,0,0,0,...,4.0,5.0,2.0,29.0,1.0,SL,AR,9.0,44%,0
4,DM de Silva,2024_T20_017,14,14,1,100.0,0,0,1,0,...,17.0,13.0,1.0,5.0,0.0,SL,AR,7.5,12%,0
5,MDKJ Perera,2024_T20_018,0,2,1,0.0,0,0,0,0,...,-0.0,1.0,0.0,0.0,0.0,SL,BAT,7.0,94%,0
6,AD Mathews,2024_T20_023,32,21,0,152.4,0,0,4,0,...,11.0,7.0,1.0,5.0,1.0,SL,AR,8.5,94%,0
7,WIA Fernando,2024_T20_023,0,8,1,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,SL,BAT,6.5,94%,0
8,S Samarawickrama,2024_T20_023,7,11,1,63.6,0,0,0,0,...,7.0,6.0,0.0,0.0,0.0,SL,WK,8.0,94%,0
9,PHKD Mendis,2024_T20_023,37,27,1,137.0,0,0,3,2,...,-0.0,-0.0,0.0,0.0,0.0,SL,BAT,7.0,94%,0


In [22]:
dfdd=totale
dfdd = pd.get_dummies(dfdd, columns=['Position'], drop_first=True, prefix=['Position'])
dfdd = dfdd[['Player','Match', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored', 'Balls Played', 'Out', 'Bat SR', '50', '100', '4s Scored', '6s Scored', 'Bat Dot%', 'Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'Bowl Avg', 'Bowl SR', '5W', '4s Given', '6s Given', 'Bowl Dot%', 'next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Credits', 'Selection', 'Points']]
dfdd['Selection'] = dfdd['Selection'].str.replace('%', '')
dfdd[dfdd.columns[6:]] = dfdd[dfdd.columns[6:]].astype(float)
df_model = dfdd[['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored','Balls Played','4s Scored', '6s Scored','Out', 'Bat SR','Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'next_runs','next_balls', 'next_overs','next_runs_given', 'next_wkts','Selection', 'Credits','Points']]
# Add new columns
df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
# Convert 'Runs Scored' column to numeric type
df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')

# Add new columns
df_model.insert(loc=8, column='30 runs', value=(df_model['Runs Scored'] > 30).astype(int))
df_model.insert(loc=9, column='50 runs', value=(df_model['Runs Scored'] > 50).astype(int))
df_model.insert(loc=10, column='100 runs', value=(df_model['Runs Scored'] > 100).astype(int))
df_model.insert(loc=11, column='Duck', value=(df_model['Runs Scored'] == 0).astype(int))
# Insert new columns into df_model based on the values of 'Wickets Taken' column
df_model.insert(loc=18, column='3 wickets', value=np.where(df_model['Wickets Taken'] >= 3, 1, 0))
df_model.insert(loc=19, column='4 wickets', value=np.where(df_model['Wickets Taken'] >= 4, 1, 0))
df_model.insert(loc=20, column='5 wickets', value=np.where(df_model['Wickets Taken'] >= 5, 1, 0))

# Calculate points based on the provided rules
df_model['Points'] = (
    # announced
    df_model['Runs Scored'] +            # run
    df_model['4s Scored'] +              # boundary_bonus
    2 * df_model['6s Scored'] +          # six_bonus
    4 * df_model['30 runs'] +            # 30_run_bonus
    8 * (df_model['50 runs'] == 1) +     # half_century_bonus
    16 * (df_model['100 runs'] == 1) +   # century_bonus
    -2 * df_model['Duck'] +              # dismissal_for_duck
    25 * df_model['Wickets Taken'] +     # wicket
    8 * (df_model['Wickets Taken'] >= 3) +  # 3_wicket_haul
    8 * (df_model['Wickets Taken'] >= 4) +  # 4_wicket_haul
    16 * (df_model['Wickets Taken'] >= 5) + # 5_wicket_haul
        # maiden
                # catch
     # runout(DirectHit/Stumping)
      # runout(Catcher/Thrower)
    ((df_model['Overs Bowled'] >= 2) & (df_model['Econ'] < 5)) * 6 +     # economy_points
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] > 170)) * 6 + # strike_rate_points
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 150.01) & (df_model['Bat SR'] <= 170)) * 4 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 130) & (df_model['Bat SR'] <= 150)) * 2 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 60) & (df_model['Bat SR'] <= 70)) * -2 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] >= 50) & (df_model['Bat SR'] <= 59.99)) * -4 +
    ((df_model['Balls Played'] >= 10) & (df_model['Bat SR'] < 50)) * -6
)

# Display the DataFrame with new 'Points' column
df_model.head()



df_model

C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\4133341787.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\4133341787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Runs Scored'] = pd.to_numeric(df_model['Runs Scored'], errors='coerce')
C:\Users\anilk\AppData\Local\Temp\ipykernel_23832\4133341787.py:23: SettingWithCopyWarning: 
A value is trying to be set 

,Player,Position_BAT,Position_BOWL,Position_WK,Runs Scored,Balls Played,4s Scored,6s Scored,30 runs,50 runs,...,4 wickets,5 wickets,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Selection,Credits,Points
0,MD Shanaka,False,False,False,7,8.0,0.0,0.0,0,0,...,0,0,9.0,9.0,0.0,0.0,0.0,3.0,8.0,7.0
1,MD Shanaka,False,False,False,9,9.0,0.0,1.0,0,0,...,0,0,2.0,8.0,0.0,0.0,0.0,3.0,8.0,11.0
2,MD Shanaka,False,False,False,2,8.0,0.0,0.0,0,0,...,0,0,31.0,24.0,1.0,10.0,1.0,3.0,8.0,2.0
3,MD Shanaka,False,False,False,31,24.0,3.0,1.0,1,0,...,0,0,50.0,44.0,0.0,0.0,0.0,3.0,8.0,65.0
4,MD Shanaka,False,False,False,50,44.0,3.0,2.0,1,0,...,0,0,8.0,14.0,0.0,0.0,0.0,3.0,8.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,Tanzim Hasan Sakib,False,True,False,0,2.0,0.0,0.0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,85.0,7.0,-2.0
355,Taijul Islam,False,True,False,5,7.0,1.0,0.0,0,0,...,0,0,6.0,3.0,0.0,0.0,0.0,59.0,7.0,6.0
356,Taijul Islam,False,True,False,6,3.0,0.0,1.0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,59.0,7.0,8.0
357,Taijul Islam,False,True,False,0,1.0,0.0,0.0,0,0,...,0,0,0.0,2.0,0.0,0.0,0.0,59.0,7.0,-2.0


In [23]:
X = df_model[df_model.columns[4:6]]
y = df_model[df_model.columns[6:7]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345)
fours_model = LinearRegression().fit(X_train, y_train)
print('Training set accuracy:', fours_model.score(X_train, y_train))
print('Test set accuracy:', fours_model.score(X_test, y_test))

Training set accuracy: 0.6944661646743613
Test set accuracy: 0.7848041110485888


In [17]:
X = df_model[df_model.columns[4:6]]
y = df_model[df_model.columns[7:8]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=999)
sixes_model = LinearRegression().fit(X_train, y_train)
print("Training set accuracy:", sixes_model.score(X_train, y_train))
print('Test set accuracy:', sixes_model.score(X_test, y_test))

Training set accuracy: 0.5761741505939932
Test set accuracy: 0.5774398659713931


In [18]:

X = df_model[df_model.columns[1:12]]
y = df_model[df_model.columns[12:13]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9999)
out_model = LogisticRegression().fit(X_train, y_train)
print('Training set accuracy:', out_model.score(X_train, y_train))
print('Test set accuracy:', out_model.score(X_test, y_test))

Training set accuracy: 0.8454545454545455
Test set accuracy: 0.8097826086956522


C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\anilk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
# Preparing current match's data in the relevant format and predicting the Dream11 points a player will score
test1 = df_model1

# Handle missing values in 'Bowl Avg' and 'Bowl SR' columns
test1['Bowl Avg'] = np.where(test1['Bowl Avg'] == '-', 0, test1['Bowl Avg'])
test1['Bowl SR'] = np.where(test1['Bowl SR'] == '', 0, test1['Bowl SR'])

# One-hot encode the 'Position' column
test1 = pd.get_dummies(test1, columns=['Position'], drop_first=True, prefix=['Position'])

# Select relevant columns for prediction
test1 = test1[['Player', 'Team', 'Match', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Runs Scored', 'Balls Played', 'Out', 'Bat SR', '50', '100', '4s Scored', '6s Scored', 'Bat Dot%', 'Overs Bowled', 'Runs Given', 'Wickets Taken', 'Econ', 'Bowl Avg', 'Bowl SR', '5W', '4s Given', '6s Given', 'Bowl Dot%', 'next_runs', 'next_balls', 'next_overs', 'next_runs_given', 'next_wkts', 'Credits', 'Selection', 'Points']]

# Clean 'Selection' column
test1['Selection'] = test1['Selection'].str.replace('%', '')

# Convert selected columns to float
test1[test1.columns[6:]] = test1[test1.columns[6:]].astype(float)

# Create a new DataFrame for prediction
test2 = test1[['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK','Runs Scored', 'Balls Played','Overs Bowled', 'Runs Given', 'Wickets Taken','next_runs', 'next_balls','next_overs','next_runs_given','next_wkts','Selection' ,'Credits']]

# Replace 'next_runs_given' with 0 if 'next_overs' is 0.0
test2['next_runs_given'] = np.where(test2['next_overs'] == 0.0, 0.0, test2['next_runs_given'])

# Create additional features''''

test2.insert(loc=6, column='30 runs', value=np.where(test2['next_runs'] > -30, 1, 0))


test2.insert(loc=7, column='50 runs', value=np.where(test2['next_runs'] >= 50, 1, 0))
test2.insert(loc=8, column='100 runs', value=np.where(test2['next_runs'] >= 100, 1, 0))
test2.insert(loc=9, column='Duck', value=np.where(test2['next_runs'] == 0, 1, 0))
test2.insert(loc=13, column='3 wickets', value=np.where(test2['next_wkts'] >= 3, 1, 0))
test2.insert(loc=14, column='4 wickets', value=np.where(test2['next_wkts'] >= 4, 1, 0))
test2.insert(loc=15, column='5 wickets', value=np.where(test2['next_wkts'] >= 5, 1, 0))
test2.insert(loc=10, column='Bat SR', value=round(test2['next_runs'] / test2['next_balls'] * 100, 1))
test2.insert(loc=14, column='Econ', value=round(test2['next_runs_given'] / test2['next_overs'], 1))
test2.insert(loc=6, column='4s Scored', value=fours_model.predict(test2[test2.columns[4:6]]))
test2.insert(loc=7, column='6s Scored', value=fours_model.predict(test2[test2.columns[4:6]]))
test2.insert(loc=12, column='Out', value=out_model.predict(test2[test2.columns[1:12]]))




test2 = test2.fillna(0)
test2 = test2.replace(np.inf, 0)
df_model.fillna(0, inplace=True)
test2['Predicted_Points'] = points_model.predict(test2[test2.columns[1:]])
test2 = pd.merge(test2, df_model1[['Player', 'Team']], on='Player', how='left')
test2 = pd.get_dummies(test2, columns=['Team'], drop_first=True, prefix=['Team'])
test2['4s Scored'] = round(test2['4s Scored'], 0)
test2['6s Scored'] = round(test2['6s Scored'], 0)

# Display the DataFrame


test2

NameError: name 'df_model1' is not defined

In [21]:
df_model2=totali

# Calculate points based on the provided rules
df_model2['Points'] = (
    # announced
    df_model1['Runs Scored'] +            # run
    df_model1['4s Scored'] +              # boundary_bonus
    2 * df_model1['6s Scored'] +          # six_bonus
    4 * df_model1['30 runs'] +            # 30_run_bonus
    8 * (df_model1['50 runs'] == 1) +     # half_century_bonus
    16 * (df_model1['100 runs'] == 1) +   # century_bonus
    -2 * df_model1['Duck'] +              # dismissal_for_duck
    25 * df_model1['Wickets Taken'] +     # wicket
    8 * (df_model1['Wickets Taken'] >= 3) +  # 3_wicket_haul
    8 * (df_model1['Wickets Taken'] >= 4) +  # 4_wicket_haul
    16 * (df_model1['Wickets Taken'] >= 5) + # 5_wicket_haul
        # maiden
                # catch
     # runout(DirectHit/Stumping)
      # runout(Catcher/Thrower)
    ((df_model1['Overs Bowled'] >= 2) & (df_model1['Econ'] < 5)) * 6 +     # economy_points
    ((df_model1['Balls Played'] >= 10) & (df_model1['Bat SR'] > 170)) * 6 + # strike_rate_points
    ((df_model1['Balls Played'] >= 10) & (df_model1['Bat SR'] >= 150.01) & (df_model1['Bat SR'] <= 170)) * 4 +
    ((df_model1['Balls Played'] >= 10) & (df_model1['Bat SR'] >= 130) & (df_model1['Bat SR'] <= 150)) * 2 +
    ((df_model1['Balls Played'] >= 10) & (df_model1['Bat SR'] >= 60) & (df_model1['Bat SR'] <= 70)) * -2 +
    ((df_model1['Balls Played'] >= 10) & (df_model1['Bat SR'] >= 50) & (df_model1['Bat SR'] <= 59.99)) * -4 +
    ((df_model1['Balls Played'] >= 10) & (df_model1['Bat SR'] < 50)) * -6
)

df_model1
    # announce

NameError: name 'df_model1' is not defined

In [21]:
test2[['Player', 'Team_SL' ,'Credits', 'Predicted_Points']].sort_values('Predicted_Points', ascending=False)


,Player,Team_SL,Credits,Predicted_Points
3,PWH de Silva,False,0.0,106.429731
23,Taskin Ahmed,False,8.0,100.985306
9,PHKD Mendis,True,7.0,99.811538
15,Liton Das,False,0.0,57.554876
26,Shoriful Islam,False,8.5,57.485308
17,Mohammad Naim,False,7.5,57.248614
16,Mahmudullah,False,8.0,57.223305
20,Mahedi Hasan,False,7.0,54.467194
11,B Fernando,True,6.5,51.341013
12,M Pathirana,True,9.0,50.409923


In [22]:
playing11 =  ["KIC Asalanka", "WIA Fernando", "BKG Mendis", "S Samarawickrama", "PHKD Mendis",  "AD Mathews", "MD Shanaka", "M Theekshana", "A Dananjaya", "B Fernando",  "M Pathirana", "Najmul Hossain Shanto", "Liton Das", "Soumya Sarkar", "Towhid Hridoy", "Jaker Ali", "Mahmudullah", "Mahedi Hasan", "Mustafizur Rahman", "Taskin Ahmed", "Shoriful Islam", "Rishad Hossain"]


test2 = test2[test2['Player'].isin(playing11)]
# Check the respective points of all players
test2


,Player,Position_BAT,Position_BOWL,Position_WK,Runs Scored,Balls Played,4s Scored,6s Scored,30 runs,50 runs,...,next_runs,next_balls,next_overs,next_runs_given,next_wkts,Selection,Credits,Predicted_Points,Team_SL,Points
0,MD Shanaka,False,False,False,13.0,8.0,1.0,1.0,1,0,...,24.0,15.0,1.0,12.0,1.0,3.0,8.0,21.791896,True,20.0
1,KIC Asalanka,True,False,False,4.0,8.0,0.0,0.0,1,0,...,8.0,11.0,0.0,0.0,0.0,8.0,8.5,6.771215,True,8.0
2,BKG Mendis,False,False,True,16.0,13.0,1.0,1.0,1,0,...,23.0,15.0,0.0,0.0,0.0,70.0,8.5,23.699120,True,27.0
6,AD Mathews,False,False,False,4.0,5.0,0.0,0.0,1,0,...,-0.0,12.0,1.0,12.0,1.0,94.0,8.5,6.355701,True,6.0
7,WIA Fernando,True,False,False,1.0,3.0,0.0,0.0,1,0,...,7.0,7.0,0.0,0.0,0.0,94.0,6.5,3.878571,True,5.0
8,S Samarawickrama,False,False,True,23.0,12.0,2.0,2.0,1,0,...,-0.0,-0.0,0.0,0.0,0.0,94.0,8.0,34.827544,True,25.0
9,PHKD Mendis,True,False,False,65.0,39.0,6.0,6.0,1,1,...,65.0,39.0,0.0,0.0,0.0,94.0,7.0,99.811538,True,99.0
10,M Theekshana,False,True,False,0.0,0.0,0.0,0.0,1,0,...,1.0,2.0,3.0,21.0,1.0,94.0,8.5,29.435002,True,29.0
11,B Fernando,False,True,False,0.0,0.0,0.0,0.0,1,0,...,0.0,1.0,0.0,0.0,1.0,94.0,6.5,51.341013,True,58.0
12,M Pathirana,False,True,False,0.0,0.0,0.0,0.0,1,0,...,0.0,0.0,2.0,12.0,2.0,94.0,9.0,50.409923,True,52.0


In [23]:
test2[['Player', 'Team_SL', 'next_runs', 'Credits', 'Predicted_Points']].sort_values('Predicted_Points', ascending=False)

,Player,Team_SL,next_runs,Credits,Predicted_Points
23,Taskin Ahmed,False,0.0,8.0,100.985306
9,PHKD Mendis,True,65.0,7.0,99.811538
15,Liton Das,False,7.0,0.0,57.554876
26,Shoriful Islam,False,0.0,8.5,57.485308
16,Mahmudullah,False,7.0,8.0,57.223305
20,Mahedi Hasan,False,0.0,7.0,54.467194
11,B Fernando,True,0.0,6.5,51.341013
12,M Pathirana,True,0.0,9.0,50.409923
8,S Samarawickrama,True,-0.0,8.0,34.827544
24,Jaker Ali,False,24.0,6.0,30.193956


In [24]:
import pulp as p
test3=pd.DataFrame()
test3=test2
Lp_prob = p.LpProblem('Problem', p.LpMaximize)
test3=test3.reset_index()

In [90]:
pip install PuLP

Note: you may need to restart the kernel to use updated packages.


NameError: name 'decision_variabltes' is not defined

In [116]:
decision_variables = [p.LpVariable(f'dv_{i}', cat='Binary') for i in range(len(test3))]
total_points = ""

for rownum, row in test3.iterrows():
    for i, j in enumerate(decision_variables):
        if rownum == i:
            formula = row['Predicted_Points'] * j
            total_points += formula
Lp_prob += total_points
batsmen = ""
for rownum, row in test3.iterrows():
    for i, j in enumerate(decision_variables):
        if rownum == i:
            formula = row['Position_BAT'] * j
            batsmen += formula
Lp_prob += batsmen >= 3
Lp_prob += batsmen <= 6

# Constraint 2: Bowlers
bowlers = ""
for rownum, row in test3.iterrows():
    for i, j in enumerate(decision_variables):
        if rownum == i:
            formula = row['Position_BOWL'] * j
            bowlers += formula
Lp_prob += bowlers >= 3
Lp_prob += bowlers <= 6

# Constraint 3: Wicket keepers
wicketkeeper = ""
for rownum, row in test3.iterrows():
    for i, j in enumerate(decision_variables):
        if rownum == i:
            formula = row['Position_WK'] * j
            wicketkeeper += formula
Lp_prob += wicketkeeper >= 1
Lp_prob += wicketkeeper <= 4

# Constraint 4: Total players
all_players = ""
for rownum, row in test3.iterrows():
    for i, j in enumerate(decision_variables):
        if rownum == i:
            formula = j
            all_players += formula
Lp_prob += all_players == 11

# Constraint 5: Credits
credits = ""
for rownum, row in test3.iterrows():
    for i, j in enumerate(decision_variables):
        if rownum == i:
            formula = j
            credits += formula
Lp_prob += credits <= 100

# Constraint 6: Max from one team
oneteam = ""
for rownum, row in test3.iterrows():
    for i, j in enumerate(decision_variables):
        if rownum == i:
            formula = row['Team_SL'] * j
            oneteam += formula
Lp_prob += oneteam >= 4
Lp_prob += oneteam <= 7

optimization_result = Lp_prob.solve()

In [141]:
selected_players_df

0     BKG Mendis
1          False
2          False
3           True
4           70.0
         ...    
83         False
84          10.0
85           8.5
86     59.938947
87         False
Length: 88, dtype: object

In [25]:
import pulp as p
import pandas as pd

# Initialize the linear programming problem
Lp_prob = p.LpProblem('Dream11_Team_Selection', p.LpMaximize)

# Decision Variables
decision_variables = {row['Player']: p.LpVariable(row['Player'], cat='Binary') for index, row in test3.iterrows()}

# Objective Function: Maximize predicted points
total_points = p.lpSum(row['Predicted_Points'] * decision_variables[row['Player']] for index, row in test3.iterrows())
Lp_prob += total_points

# Constraints
# Minimum 3 and maximum 6 batsmen
batsmen_constraint = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Position_BAT'] == 1)
Lp_prob += batsmen_constraint >= 3
Lp_prob += batsmen_constraint <= 6

# Minimum 3 and maximum 6 bowlers
bowlers_constraint = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Position_BOWL'] == 1)
Lp_prob += bowlers_constraint >= 3
Lp_prob += bowlers_constraint <= 6

# Minimum 1 and maximum 4 wicketkeepers
wicketkeepers_constraint = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Position_WK'] == 1)
Lp_prob += wicketkeepers_constraint >= 1
Lp_prob += wicketkeepers_constraint <= 4

# 11 players
Lp_prob += p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows()) == 11

# Sum of credits to be within 100
credits_constraint = p.lpSum(row['Credits'] * decision_variables[row['Player']] for index, row in test3.iterrows())
Lp_prob += credits_constraint <= 100

# Minimum 4 and maximum 7 players from one team
team_constraints = {}
for team in set(test3['Team_SL']):
    team_constraints[team] = p.lpSum(decision_variables[row['Player']] for index, row in test3.iterrows() if row['Team_SL'] == team)
    Lp_prob += team_constraints[team] >= 4
    Lp_prob += team_constraints[team] <= 7

# Solve the optimization problem
optimization_result = Lp_prob.solve()

# Create a DataFrame for selected players
selected_players_df = pd.concat([test3.loc[test3['Player'] == player, ['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Selection', 'Credits', 'Predicted_Points', 'Team_SL']] for player in decision_variables if p.value(decision_variables[player]) == 1], ignore_index=True)

# Assign values based on the predicted points
selected_players_df['value'] = selected_players_df['Predicted_Points'].apply(lambda x: 2.0 if x == selected_players_df['Predicted_Points'].max() else (1.5 if x == selected_players_df['Predicted_Points'].nlargest(2).iloc[-1] else 1.0))

# Print the selected players DataFrame
print("Selected Players:")
selected_players_df

Selected Players:


,Player,Position_BAT,Position_BOWL,Position_WK,Selection,Credits,Predicted_Points,Team_SL,value
0,S Samarawickrama,False,False,True,94.0,8.0,34.827544,True,1.0
1,PHKD Mendis,True,False,False,94.0,7.0,99.811538,True,1.5
2,B Fernando,False,True,False,94.0,6.5,51.341013,True,1.0
3,M Pathirana,False,True,False,94.0,9.0,50.409923,True,1.0
4,Liton Das,False,False,False,0.0,0.0,57.554876,False,1.0
5,Mahmudullah,True,False,False,68.0,8.0,57.223305,False,1.0
6,Najmul Hossain Shanto,True,False,False,85.0,7.5,24.070949,False,1.0
7,Mahedi Hasan,False,False,False,59.0,7.0,54.467194,False,1.0
8,Taskin Ahmed,False,True,False,42.0,8.0,100.985306,False,2.0
9,Jaker Ali,False,False,True,4.0,6.0,30.193956,False,1.0


In [26]:
selected_players_df[['Player', 'Position_BAT','Position_BOWL','Position_WK'	,'Selection','Credits','Predicted_Points','Team_SL','value']].sort_values('Predicted_Points', ascending=False)

,Player,Position_BAT,Position_BOWL,Position_WK,Selection,Credits,Predicted_Points,Team_SL,value
8,Taskin Ahmed,False,True,False,42.0,8.0,100.985306,False,2.0
1,PHKD Mendis,True,False,False,94.0,7.0,99.811538,True,1.5
4,Liton Das,False,False,False,0.0,0.0,57.554876,False,1.0
10,Shoriful Islam,False,True,False,10.0,8.5,57.485308,False,1.0
5,Mahmudullah,True,False,False,68.0,8.0,57.223305,False,1.0
7,Mahedi Hasan,False,False,False,59.0,7.0,54.467194,False,1.0
2,B Fernando,False,True,False,94.0,6.5,51.341013,True,1.0
3,M Pathirana,False,True,False,94.0,9.0,50.409923,True,1.0
0,S Samarawickrama,False,False,True,94.0,8.0,34.827544,True,1.0
9,Jaker Ali,False,False,True,4.0,6.0,30.193956,False,1.0


In [27]:
# Total Predicted Points
(selected_players_df['Predicted_Points'] * selected_players_df['value']).sum()

# Total credits utilized


769.2619862638727

In [28]:
# Total Predicted Points

# Total credits utilized
selected_players_df['Credits'].sum()

# Batsmen selected


75.5

In [29]:
selected_players_df[selected_players_df["Position_BAT"] == 1]['Player']



1              PHKD Mendis
5              Mahmudullah
6    Najmul Hossain Shanto
Name: Player, dtype: object

In [152]:

# Bowler selected
selected_players_df[selected_players_df["Position_BOWL"] == 1]['Player']

# All-rounder selected


3       M Theekshana
8       Taskin Ahmed
10    Shoriful Islam
Name: Player, dtype: object

In [153]:
selected_players_df[(selected_players_df['Position_BAT']) & (selected_players_df['Position_BOWL']) & (selected_players_df['Position_WK'])]['Player']

# Wicket-keeper selected


Series([], Name: Player, dtype: object)

In [154]:
selected_players_df[selected_players_df['Position_WK'] == 1]["Player"]

# Team split


0          BKG Mendis
1    S Samarawickrama
9           Jaker Ali
Name: Player, dtype: object

In [30]:
selected_players_df["Team_SL"].value_counts()

Team_SL
False    7
True     4
Name: count, dtype: int64

In [161]:
print(selected_players_df.columns)

Index(['Player', 'Position_BAT', 'Position_BOWL', 'Position_WK', 'Selection',
       'Credits', 'Predicted_Points', 'Team_SL', 'value'],
      dtype='object')


In [162]:
print(selected_players_df.head())


             Player  Position_BAT  Position_BOWL  Position_WK  Selection  \
0        BKG Mendis         False          False         True       70.0   
1  S Samarawickrama         False          False         True       94.0   
2       PHKD Mendis          True          False        False       94.0   
3      M Theekshana         False           True        False       94.0   
4         Liton Das         False          False        False        0.0   

   Credits  Predicted_Points  Team_SL  value  
0      8.5         25.157473     True    1.0  
1      8.0         35.459080     True    1.0  
2      7.0         94.412713     True    1.5  
3      8.5         30.896440     True    1.0  
4      0.0         57.338923    False    1.0  


In [32]:
from itertools import permutations

# Final List of players with recommendations for captain and vice captain
result2 = pd.DataFrame(list(permutations(selected_players_df['value'].values, 11)), columns=selected_players_df['Player'].values).drop_duplicates().T.add_prefix('Combination')
result2 = pd.merge(selected_players_df[['Player', 'Predicted_Points']], result2, left_on='Player', right_index=True, how='left')
result3 = result2[result2.columns[2:]].multiply(result2['Predicted_Points'], axis='index')
combinations = result3[result3.columns[1:]].sum(axis=0, skipna=True).sort_values(ascending=False).head(20).reset_index()['index'] 
final_combinations = result2.loc[:, result2.columns.isin(list(combinations))] 
final_combinations.insert(loc=0, column='Player', value= result2['Player'])
final_combinations.insert(loc=1, column='Predicted_Points', value= result2['Predicted_Points'])
final_combinations.columns = ['Player', 'Predicted_Points', 'Combination1', 'Combination2', 'Combination3', 'Combination4', 'Combination5', 'Combination6', 'Combination7', 'Combination8', 'Combination9', 'Combination10', 'Combination11', 'Combination12', 'Combination13', 'Combination14', 'Combination15', 'Combination16', 'Combination17', 'Combination18', 'Combination19', 'Combination20']
final_combinations


,Player,Predicted_Points,Combination1,Combination2,Combination3,Combination4,Combination5,Combination6,Combination7,Combination8,...,Combination11,Combination12,Combination13,Combination14,Combination15,Combination16,Combination17,Combination18,Combination19,Combination20
0,S Samarawickrama,34.827544,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.5
1,PHKD Mendis,99.811538,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0
2,B Fernando,51.341013,1.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,M Pathirana,50.409923,1.0,1.5,1.0,1.0,1.0,1.0,1.0,1.0,...,1.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Liton Das,57.554876,1.0,1.0,1.5,1.0,1.0,1.0,1.0,1.0,...,1.0,1.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,Mahmudullah,57.223305,1.0,1.0,1.0,1.5,1.0,1.0,1.0,1.0,...,1.0,1.0,1.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,Najmul Hossain Shanto,24.070949,1.0,1.0,1.0,1.0,1.5,1.0,1.0,1.0,...,1.0,1.0,1.0,1.5,1.0,1.0,1.0,1.0,1.0,1.0
7,Mahedi Hasan,54.467194,1.0,1.0,1.0,1.0,1.0,1.5,1.0,1.0,...,1.0,1.0,1.0,1.0,1.5,1.0,1.0,1.0,1.0,1.0
8,Taskin Ahmed,100.985306,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.5,1.0,1.0,2.0,1.0
9,Jaker Ali,30.193956,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.0,1.0,1.0
